In [33]:
import glob, json, pandas as pd
df1 = pd.read_csv('../data/coco_degradation/val2017.csv')

df1.head(2)

,image,quality,width,height,channels,mean_squared_error,peak_signal_noise_ratio,structural_similarity
0,000000000139.jpg,1,640,426,3,454.463,21.556,0.600
1,000000000285.jpg,1,586,640,3,781.224,19.203,0.345


In [34]:
df1.channels.value_counts()

3    499000
1      1000
Name: channels, dtype: int64

In [35]:
def load(f):
    with open(f) as jsf:
        return json.load(jsf)

In [9]:
jsons = glob.glob('../results/*/filesizes*.json')
objs = [load(f) for f in jsons]

In [46]:
def dicts(objs):
    for obj in objs:
        q = obj['quality']
        if q == 'original' or q == 0:
            continue
        for img, size in obj['filesizes'].items():
            yield dict(
                quality=q,
                image=img,
                filesize=size,
            )

In [47]:
df2 = pd.DataFrame(dicts(objs))

In [48]:
# unique img<->Q
df2.drop('filesize', 1).value_counts().value_counts()

1    500000
dtype: int64

In [49]:
idf1 = df1.set_index(['image', 'quality'])
idf2 = df2.set_index(['image', 'quality'])

In [50]:
idf1['filesize'] = idf2.filesize

In [52]:
idf1.head(2)

,,width,height,channels,mean_squared_error,peak_signal_noise_ratio,structural_similarity,filesize
image,quality,,,,,,,
000000000139.jpg,1,640,426,3,454.463,21.556,0.600,4748
000000000285.jpg,1,586,640,3,781.224,19.203,0.345,10097


In [59]:
df = idf1.reset_index()
df.to_csv(
    '../data/val2017_degraded.csv',
    float_format='%.3f'
)

Mean values per quality  
(TODO)

In [58]:
df.drop('channels', 1).groupby('quality').mean()

,width,height,mean_squared_error,peak_signal_noise_ratio,structural_similarity,filesize
quality,,,,,,
1,573.7548,483.543,536.180397,21.204653,0.538857,7203.7820
2,573.7548,483.543,535.909813,21.206728,0.539011,7207.2246
3,573.7548,483.543,506.466772,21.449087,0.557630,7596.0948
4,573.7548,483.543,412.602897,22.409323,0.596385,8406.9092
5,573.7548,483.543,340.629869,23.328939,0.626999,9321.4490
...,...,...,...,...,...,...
96,573.7548,483.543,0.488821,54.827485,0.998574,171089.2448
97,573.7548,483.543,1.103242,48.541253,0.996691,182717.3818
98,573.7548,483.543,0.365069,52.839577,0.998546,191785.5742
